# LoC image analysis

This notebook is designed to take the raw output of a LoC data set, monolayer or bilayer, and:
 (trying to check if dask multichan works)

5. Localise and measure properties
6. Unite the localisations over the z-range
7. Save out the Z-tracks
8. Extract the maximum intensity from each cell

In [1]:
import os
import glob
from octopusheavy import DaskOctopusHeavyLoader
import napari
from skimage.io import imshow,  imsave, imread
import napari
import btrack
from tqdm.auto import tqdm
import dask.array as da

/home/dayn/miniconda3/envs/btrack_test/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading images

Define root path and individual experiment IDs

In [2]:
root_path = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy'

In [3]:
expt_IDs = ['co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/images/',
            'co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-VWF-iCAM1-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_262_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_130_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_253_/images/'
           ]

## Check to see what channels each expt has

In [4]:
for i, expt in enumerate(expt_IDs):
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    if 'MASK1' in [channel.name for channel in images.channels]:
        images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
        print(i, images.channels)

0 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
1 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
2 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
3 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
4 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
5 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]


In [5]:
images['CH1']

dask.array<stack, shape=(81, 2304, 2304), dtype=uint16, chunksize=(1, 2304, 2304), chunktype=numpy.ndarray>

### Defining properties to measure similarities of z-slices

In [6]:
props = ('axis_major_length', 
         'axis_minor_length', 
         'eccentricity', 
         'area', 
         #'intensity_image', 
         'orientation',
         'mean_intensity')

# I think if this one doesn't work it's due to the filename... but surely because I'm reading images from that address then the filepath works...?

In [7]:
for i, expt in tqdm(enumerate(expt_IDs), total = len(expt_IDs)):
    props = ('axis_major_length', 
         'axis_minor_length', 
         'eccentricity', 
         'area', 
         #'intensity_image', 
         'orientation',
         'mean_intensity')
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    multichannel_stack = da.stack([images['CH1'], images['CH2'], images['CH3'], images['CH4']], axis = -1)
    if 'MASK1' in [channel.name for channel in images.channels]: 
        ### create filename for tracks and objects 
        dirname = os.path.dirname(images.files('CH1')[0]).replace('images','')
        objects_fn = os.path.join(dirname, 'objects.hdf5')
        tracks_fn = os.path.join(dirname, 'tracks.hdf5')
        ### try saving locally instead of online
        tracks_fn = tracks_fn.split('Light microscopy/')[-1].replace('/', '_')
        objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
            use_weighted_centroid = False
        )
        ### prune objects 
        objects = [o for o in objects if o.properties['area']>50]
#         ### save out objects
#         with btrack.dataio.HDF5FileHandler(
#              objects_fn, 'w', obj_type='obj_type_1',
#         ) as hdf:
#             hdf.write_segmentation(images['MASK1'])
#             hdf.write_objects(objects)
        ### redefine properties as multichannel image was measured
        props = ('axis_major_length', 
             'axis_minor_length', 
             'eccentricity', 
             'area', 
             #'intensity_image', 
             'orientation',
             'mean_intensity-0',
             'mean_intensity-1',
             'mean_intensity-2',
             'mean_intensity-3',)
        
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:
            # configure the tracker using a config file
            tracker.configure('/home/dayn/analysis/BayesianTracker/models/particle_config.json')
            tracker.verbose = True
            ### set max search radius
            tracker.max_search_radius = 100
            # use visual features to track
            tracker.features = props
            # append the objects to be tracked
            tracker.append(objects)
            # set the volume (Z axis volume limits default to [-1e5, 1e5] for 2D data)
            tracker.volume=((0, 2304), (0, 2304), (-1e5, 1e5))
            # track them (in interactive mode)
#             tracker.track_interactive(step_size=100)
            tracker.track(tracking_updates =['visual', 'motion'], step_size=10)
            # generate hypotheses and run the global optimizer
            tracker.optimize()
            # get the tracks as a python list
            tracks = tracker.tracks
#             # filter tracks
#             tracks = [track for track in tracks if len(track) >= 3]
            # optional: get the data in a format for napari
            #   data, properties, graph = tracker.to_napari()
            tracker.export(tracks_fn, obj_type = 'obj_type_1')
    break

  0%|                                                                            | 0/6 [00:00<?, ?it/s][INFO][2023/01/10 10:33:48 AM] Localizing objects from segmentation...
[INFO][2023/01/10 10:33:48 AM] Found intensity_image data
[INFO][2023/01/10 10:35:57 AM] Objects are of type: <class 'dict'>
[INFO][2023/01/10 10:35:57 AM] ...Found 30362 objects in 68 frames.
[INFO][2023/01/10 10:35:57 AM] Loaded btrack: /home/dayn/analysis/BayesianTracker_/btrack/libs/libtracker.so
[INFO][2023/01/10 10:35:57 AM] btrack (v0.5.0) library imported
[INFO][2023/01/10 10:35:57 AM] Starting BayesianTracker session
[INFO][2023/01/10 10:35:57 AM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/particle_config.json
[INFO][2023/01/10 10:35:57 AM] Setting max_search_radius -> 100
[INFO][2023/01/10 10:35:57 AM] Setting features -> ('axis_major_length', 'axis_minor_length', 'eccentricity', 'area', 'orientation', 'mean_intensity-0', 'mean_intensity-1', 'mean_intensity-2', 'mean_intensity-

GLPK Integer Optimizer 5.0
16212 rows, 12600 columns, 17094 non-zeros
12600 integer variables, all of which are binary
Preprocessing...
8106 rows, 12600 columns, 17094 non-zeros
12600 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 8106
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
8106 rows, 12600 columns, 17094 non-zeros
*     0: obj =   5.291086383e+04 inf =   0.000e+00 (1665)
Perturbing LP to avoid stalling [741]...
Removing LP perturbation [1645]...
*  1645: obj =   4.357230139e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  1645: mip =     not found yet >=              -inf        (1; 0)
+  1645: >>>>>   4.357230139e+04 >=   4.357230139e+04   0.0% (1; 0)
+  1645: mip =   4.357230139e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL

[INFO][2023/01/10 10:37:14 AM] Writing tracks/obj_type_1
[WARNING][2023/01/10 10:37:14 AM] Removing tracks/obj_type_1.
[INFO][2023/01/10 10:37:14 AM] Writing dummies/obj_type_1
[INFO][2023/01/10 10:37:14 AM] Writing LBEP/obj_type_1
[INFO][2023/01/10 10:37:14 AM] Writing fates/obj_type_1
[INFO][2023/01/10 10:37:14 AM] Closing HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.h5
[INFO][2023/01/10 10:37:14 AM] Ending BayesianTracker session
  0%|                                                                            | 0/6 [03:26<?, ?it/s]


In [12]:
objects[0]

,ID,x,y,z,t,dummy,states,label,axis_major_length,axis_minor_length,eccentricity,area,orientation,mean_intensity-0,mean_intensity-1,mean_intensity-2,mean_intensity-3
0,0,388.922414,3.62931,0.0,0,False,7,5,21.899223,6.872596,0.94948,116,-1.485736,101.060345,106.413793,106.784483,111.672414


# It won't let me save tracks to the server? But will let me save locally

In [8]:
tracks_fn

'co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.hdf5'

In [9]:
with btrack.dataio.HDF5FileHandler("tracks.h5", "w", obj_type="obj_type_1") as hdf:
    hdf.write_tracks(tracks)

[INFO][2023/01/10 10:37:47 AM] Opening HDF file: tracks.h5...
[INFO][2023/01/10 10:37:47 AM] Writing objects/obj_type_1
[INFO][2023/01/10 10:37:47 AM] Writing labels/obj_type_1
[INFO][2023/01/10 10:37:47 AM] Loading objects/obj_type_1 (29045, 5) (29045 filtered: None)
[INFO][2023/01/10 10:37:47 AM] Writing properties/obj_type_1/axis_major_length (29045,)
[INFO][2023/01/10 10:37:47 AM] Writing properties/obj_type_1/axis_minor_length (29045,)
[INFO][2023/01/10 10:37:47 AM] Writing properties/obj_type_1/eccentricity (29045,)
[INFO][2023/01/10 10:37:47 AM] Writing properties/obj_type_1/area (29045,)
[INFO][2023/01/10 10:37:47 AM] Writing properties/obj_type_1/orientation (29045,)
[INFO][2023/01/10 10:37:47 AM] Writing properties/obj_type_1/mean_intensity-0 (29045,)
[INFO][2023/01/10 10:37:47 AM] Writing properties/obj_type_1/mean_intensity-1 (29045,)
[INFO][2023/01/10 10:37:47 AM] Writing properties/obj_type_1/mean_intensity-2 (29045,)
[INFO][2023/01/10 10:37:47 AM] Writing properties/obj_

In [ ]:
with btrack.dataio.HDF5FileHandler('tracks.h5', "r", obj_type="obj_type_1") as hdf:
    tracks = hdf.tracks

[INFO][2023/01/10 10:42:26 AM] Opening HDF file: tracks.h5...
[INFO][2023/01/10 10:42:26 AM] Loading tracks/obj_type_1
[INFO][2023/01/10 10:42:26 AM] Loading LBEP/obj_type_1
[INFO][2023/01/10 10:42:26 AM] Loading objects/obj_type_1 (29045, 5) (29045 filtered: None)
[INFO][2023/01/10 10:42:26 AM] Closing HDF file: tracks.h5


In [10]:
import inspect
inspect.getsourcelines(btrack.dataio) 

(['from __future__ import annotations\n',
  '\n',
  'import csv\n',
  'import itertools\n',
  'import logging\n',
  'import os\n',
  'import re\n',
  'from functools import wraps\n',
  'from typing import TYPE_CHECKING, Any, Dict, List, Optional, Union\n',
  '\n',
  'import h5py\n',
  'import numpy as np\n',
  '\n',
  '# import core\n',
  'from . import btypes, constants, utils\n',
  '\n',
  'if TYPE_CHECKING:\n',
  '    from . import BayesianTracker\n',
  '\n',
  '# get the logger instance\n',
  'logger = logging.getLogger(__name__)\n',
  '\n',
  '\n',
  '# Choose a subset of classes/functions to document in public facing API\n',
  '__all__ = ["import_CSV"]\n',
  '\n',
  '\n',
  'def localizations_to_objects(\n',
  '    localizations: Union[\n',
  '        np.ndarray, List[btypes.PyTrackObject], Dict[str, Any]\n',
  '    ]\n',
  ') -> List[btypes.PyTrackObject]:\n',
  '    """Take a numpy array or pandas dataframe and convert to PyTrackObjects.\n',
  '\n',
  '    Parameters\n',
  '   

In [12]:
tracks[1]

,ID,t,x,y,z,parent,root,state,generation,dummy,mean_intensity-1,mean_intensity-3,orientation,mean_intensity-2,eccentricity,axis_minor_length,mean_intensity-0,axis_major_length,area
0,3,1,1881.116694,1398.286872,0.000000,3,3,5,0,False,115.873582,129.222042,-1.419839,115.439222,0.557116,25.672430,114.231767,30.914445,617.0
1,3,2,1881.116694,1398.286872,0.000000,3,3,5,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,1902.144385,1427.008913,0.000000,3,3,5,0,False,126.702317,132.212121,0.001484,119.775401,0.524600,24.932157,114.181818,29.285477,561.0
3,3,4,1921.829031,1452.464321,3.913878,3,3,5,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5,1949.364516,1502.100000,0.000000,3,3,5,0,False,123.306452,142.632258,-1.175582,121.003226,0.286746,19.490489,111.670968,20.344835,310.0
5,3,6,1950.565789,1502.868421,0.000000,3,3,5,0,False,125.842105,150.046053,0.911112,123.006579,0.567245,17.867698,111.911184,21.695965,304.0
6,3,7,1950.313783,1501.498534,0.000000,3,3,5,0,False,129.697947,159.407625,0.982844,127.190616,0.347162,20.193609,112.571848,21.532842,341.0
7,3,8,1949.082707,1501.135338,0.000000,3,3,5,0,False,134.740602,159.650376,0.203720,132.263158,0.484165,17.276162,113.300752,19.744708,266.0
8,3,9,1949.546053,1500.888158,0.000000,3,3,5,0,False,143.628289,179.509868,-0.052578,140.437500,0.339176,19.129557,114.161184,20.334950,304.0
9,3,10,1950.037500,1501.360000,0.000000,3,3,5,0,False,160.435000,243.707500,0.137396,155.027500,0.309445,22.110774,116.652500,23.252045,400.0


In [14]:
btrack??

In [16]:
import inspect
inspect.getsourcelines(btrack.dataio) 

(['from __future__ import annotations\n',
  '\n',
  'import csv\n',
  'import itertools\n',
  'import logging\n',
  'import os\n',
  'import re\n',
  'from functools import wraps\n',
  'from typing import TYPE_CHECKING, Any, Dict, List, Optional, Union\n',
  '\n',
  'import h5py\n',
  'import numpy as np\n',
  '\n',
  '# import core\n',
  'from . import btypes, constants, utils\n',
  '\n',
  'if TYPE_CHECKING:\n',
  '    from . import BayesianTracker\n',
  '\n',
  '# get the logger instance\n',
  'logger = logging.getLogger(__name__)\n',
  '\n',
  '\n',
  '# Choose a subset of classes/functions to document in public facing API\n',
  '__all__ = ["import_CSV"]\n',
  '\n',
  '\n',
  'def localizations_to_objects(\n',
  '    localizations: Union[\n',
  '        np.ndarray, List[btypes.PyTrackObject], Dict[str, Any]\n',
  '    ]\n',
  ') -> List[btypes.PyTrackObject]:\n',
  '    """Take a numpy array or pandas dataframe and convert to PyTrackObjects.\n',
  '\n',
  '    Parameters\n',
  '   

#### testing unique fn 

In [49]:
tracks_fn.split('Light microscopy/')[-1].replace('/', '_')

'co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.hdf5'

In [37]:
tracks_fn

'/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.hdf5'

In [11]:
test_tracks_fn = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5'

In [12]:
with btrack.dataio.HDF5FileHandler(test_tracks_fn, "w", obj_type="obj_type_1") as hdf:
    hdf.write_tracks(tracks)

[INFO][2023/01/10 10:38:28 AM] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5...
[INFO][2023/01/10 10:38:28 AM] Writing objects/obj_type_1
[INFO][2023/01/10 10:38:28 AM] Writing labels/obj_type_1
[INFO][2023/01/10 10:38:29 AM] Loading objects/obj_type_1 (29045, 5) (29045 filtered: None)
[INFO][2023/01/10 10:38:29 AM] Writing properties/obj_type_1/axis_major_length (29045,)
[INFO][2023/01/10 10:38:29 AM] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5


OSError: [Errno 95] Unable to extend file properly, errno = 95, error message = 'operation not supported' (file write failed: time = Tue Jan 10 10:38:29 2023
, filename = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5', file descriptor = 59, errno = 95, error message = 'Operation not supported', buf = 0x890b058, total write size = 3368, bytes this sub-write = 3368, bytes actually written = 18446744073709551615, offset = 0)

## Checking tracks prior to plotting single cell info

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()

viewer.add_image(ch0, colormap='blue', blending = 'additive')
viewer.add_image(ch3, colormap= 'gray', blending = 'additive')
viewer.add_image(ch1, colormap='red', blending = 'additive')
viewer.add_image(ch2, colormap= 'green', blending = 'additive')

viewer.add_labels(mask_stack)

viewer.add_tracks(ch1_data)
viewer.add_tracks(ch2_data)
